## Import Data & Necesary Packages

In [ ]:
import os.path
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
import datetime
import matplotlib.pyplot as plt
import warnings
import math
import glob
from scipy.interpolate import interp1d
warnings.filterwarnings("ignore")
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
node = '16'
path = # Removed for confidentiality
node_16 = pd.read_csv(path, names= ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'])
node_16.head(5)

## Data Processing

In [ ]:
# Create stopbar dataframe 
stop= node_16.loc[node_16['0']=='LCS']
stop = stop.rename(columns={"0":"ID","1": "LegNo", "2":"LaneNo", "3":"LaneType"
                       ,"4":"Time_event", "5":"BinEncPhases", '6':"SPValid"
                           ,"7":"SP","8":"TimeafterPhStrt", "9":"Time2PhEnd"
                           ,"10":"Occupancy", '11':"Speed"})
stop.head()

In [ ]:
# Create adv detector dataframe 
adv = node_16.loc[node_16['0']=='LCA']
adv = adv.rename(columns={"0":"ID","1": "LegNo", "2":"LaneNo", "3":"LaneType"
                       ,"4":"Time_event", "5":"BinEncPhases", '6':"SPValid"
                           ,"7":"SP","8":"TimeafterPhStrt", "9":"Time2PhEnd"
                           ,"10":"Occupancy", '11':"Speed"})
adv.head()

In [ ]:
# Create signal phase dataframe from given SP times
sp = node_16.loc[(node_16['0']=='SP')]
sp = sp.rename(columns={"1":"Validity","2":"TimeEvent","4":"CycleNo","5":"BarrierNo","6":"NoRings"})
sp = sp.loc[sp['Validity']==1]
sp.head()

In [ ]:
# AP for active phase
sp['AP1'] = np.zeros(len(sp))
sp['AP2'] = np.zeros(len(sp))

In [ ]:
# All NoRings are 2
sp.loc[sp['NoRings']!=2]

In [ ]:
# All signal phases have 2 barriers
sp.loc[(sp['BarrierNo']!=2)&(sp['BarrierNo']!=1)]

In [ ]:
# Since all NoRings are 2, active phase 1 is column 7, active phase 2 is column 8
sp.loc[sp['NoRings'] == 2, 'AP1'] = sp.loc[sp['NoRings'] == 2, '7']
sp.loc[sp['NoRings'] == 2, 'AP2'] = sp.loc[sp['NoRings'] == 2, '8']
sp.head()

In [ ]:
# PhaseNo indicates column no. that has active phase. Depends on BarrierNo.
# PhaseState is the state of active phase. (1 – yellow, 2 – green, 0 - red)
sp['AP'] = sp.apply(lambda x: (x['7'], x['8']) if x['NoRings']==2 else x['7'], axis=1)
sp['PhaseNo'] = sp.apply(lambda x: (x['AP1']+10, x['AP2']+14) if x['BarrierNo']==2 else (x['AP1']+8, x['AP2']+12), axis=1)
sp['PhaseState'] = sp.apply(lambda x: (x[str(int(x['PhaseNo'][0]))], x[str(int(x['PhaseNo'][1]))]), axis=1)

In [ ]:
sp.head()

# Extracting Red/Green Intervals

In [ ]:
# Reference direction = eastbound (Barrier 1, Ring 1, Phase 2)
b2 = sp.loc[(sp['BarrierNo']==1)&(sp['AP1']==2)]
b2.head()

In [ ]:
# When PhaseState changes is when red turns to green or vice versa. Delta TimeEvent between events where state doesn't 
# change is on the magnitude of ~2-3 seconds.

# Barrier 1, Ring 1, Phase 2 State
b2['Phase2St'] = b2.apply(lambda x:x['PhaseState'][0], axis=1)
b2['StBefore'] = b2['PhaseState'].shift(1)
b2['StBefore'].iloc[0] = (2,2)
b2['Phase2Diff'] = b2['Phase2St'].diff()

# Negative Phase2Diff is the row where state decreased, 0 means red so that means time at which light changed to red
b2['RedStart'] = b2.apply(lambda x: x['TimeEvent'] if ((x['Phase2Diff']<0)&(x['Phase2St']==0)) else 0, axis=1)
# If Phase2Diff >= 1 AND State of one before is 0, means green phase started
b2['GreenStart'] = b2.apply(lambda x: x['TimeEvent'] if ((x['Phase2Diff']>=1)&(x['StBefore'][0]==0)) else 0, axis=1)
b2 = b2.drop(columns=['Phase2St','Phase2Diff', 'StBefore'])
# If negative, red phase started. If positive, green phase started.
b2['Green-Red'] = b2['GreenStart']-b2['RedStart']
b2.iloc[40:90]

In [ ]:
# To find all green/red intervals
time_tbl = b2.iloc[b2['Green-Red'].nonzero()[0]]

In [ ]:
# Last red interval of time_tbl has nan - ignored in calculation (boundary: very last phase)
time_tbl['R'] = time_tbl['RedStart'].shift(-1)
time_tbl['G'] = time_tbl['GreenStart'].shift(-1)
time_tbl['Red_int'] = time_tbl.apply(lambda x: (x['RedStart'],x['G']) if x['Green-Red']<0 else 0, axis=1)
time_tbl['Green_int'] = time_tbl.apply(lambda x: (x['GreenStart'],x['R']) if x['Green-Red']>0 else 0, axis=1)
time_tbl.tail()

In [ ]:
# Extracting green and red intervals from time_tbl
green_ints = time_tbl.iloc[time_tbl['Green_int'].nonzero()]['Green_int']
red_ints = time_tbl.iloc[time_tbl['Red_int'].nonzero()]['Red_int']

In [ ]:
# Creating dataframes of red/green intervals
red_int_df = red_ints.apply(pd.Series)
red_int_df.columns=['start','end']
green_int_df = green_ints.apply(pd.Series)
green_int_df.columns=['start','end']
# Last row may have NaN b/c it's a bounday
green_int_df = green_int_df[:-1]

In [ ]:
# Extract hours of green start times to see distribution of data
green_int_df['time'] = green_int_df['start'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)))
green_int_df['Hour'] = green_int_df['time'].apply(lambda x: x.hour)
green_int_df['endtime'] = green_int_df['end'].apply(lambda x: datetime.datetime.utcfromtimestamp(int(x)))
green_int_df['endHour'] = green_int_df['endtime'].apply(lambda x: x.hour)
green_int_df = green_int_df.replace([0, 1, 2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23], 
                  [17, 18 ,19, 20, 21,22,23,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16])

In [ ]:
# Distribution of green_interval start times (PST)
sns.distplot(green_int_df['Hour'], bins = 24);

# Delay Calculation

In [ ]:
df= green_int_df
df = df.reset_index()
df.head()

In [ ]:
# Each phase is about ~50-60 seconds
df['length'] = df['end']-df['start']
df.iloc[180:190]

In [ ]:
# Indices where the starting hour changes
idx_hr = df['Hour'].diff()[df['Hour'].diff() != 0].index.values
idx_hr = np.append(idx_hr,len(df)-1)
idx_hr

In [ ]:
# Final Delay Calculation Method
# For multiple intervals across different hours
avg_delay_hrs = np.array([])
prev_idx = 0
# Looping through each hour
for idx in idx_hr:
    hour = str(df.iloc[idx]['Hour'])
    delays = np.array([])
    # Looping through each green interval in the hour
    for i in np.delete(np.arange(idx),np.arange(prev_idx)):
        in_queue = 0
        adv_red = 0
        # If spans more than 1 hour, do not include in delay calculation
        if ((np.abs(df.iloc[i]['endHour'] - df.iloc[i]['Hour']) >1) or (df.iloc[i]['length']>2500)):
            continue
            
        # Number of vehicles that passed the stop bar detector during first green interval
        temp = stop.loc[(stop['Time_event']>=df['start'].iloc[i])&(stop['Time_event']<=df['end'].iloc[i])]
        if len(temp)!= 0:
            temp['diff'] = temp.apply(lambda x: x['Time_event']-df['start'].iloc[i], axis=1)
            temp_sort = temp.sort_values(by='diff')
            temp_sort = temp_sort.loc[(temp_sort['SP']!=0)&(temp_sort['LaneNo']!=1)&(temp_sort['LaneNo']!=5)
                                      & (temp_sort['LegNo']==4)& (temp_sort['LaneType']==1)& (temp_sort['SPValid']==1)]
            passed_stop = len(temp_sort)

        # Same thing but for advance detectors
        temp1 = adv.loc[(adv['Time_event']>=df['start'].iloc[i])&(adv['Time_event']<=df['end'].iloc[i])]
        if len(temp1)!=0:
            temp1['diff'] = temp1.apply(lambda x: x['Time_event']-df['start'].iloc[i], axis=1)
            temp_sort1 = temp1.sort_values(by='diff')
            temp_sort1 = temp_sort1.loc[(temp_sort1['SP']!=0)&(temp_sort1['LaneNo']!=1)&(temp_sort1['LaneNo']!=5)
                                        & (temp_sort1['LegNo']==4)&(temp_sort1['LaneType']==1)& (temp_sort1['SPValid']==1)] 
            passed_adv = len(temp_sort1)
            in_queue = passed_adv-passed_stop
            if in_queue<0:
                in_queue = 0
        
        # No. of vehicles that passed the advance detector during red interval
        advred = adv.loc[(adv['Time_event']>=df['end'].iloc[i])&(adv['Time_event']<=df['start'].iloc[i+1])]
        if len(advred)!=0:
            advred['diff'] = advred.apply(lambda x:x['Time_event'] - df['end'].iloc[i], axis=1)
            advred_sort = advred.sort_values(by='diff')
            advred_sort = advred_sort.loc[(advred_sort['SP']==0)&(advred_sort['LaneNo']!=1)&(advred_sort['LaneNo']!=5)
                                          & (advred_sort['LegNo']==4)&(advred_sort['LaneType']==1)
                                          & (advred_sort['SPValid']==1)] 
            adv_red = len(advred_sort)
            in_queue = in_queue + adv_red

        # Sorted stop detector array for next green interval
        temp2 = stop.loc[(stop['Time_event']>=df['start'].iloc[i+1])&(stop['Time_event']<=df['end'].iloc[i+1])]
        if len(temp2)!=0:
            temp2['diff'] = temp2.apply(lambda x: x['Time_event']-df['start'].iloc[i+1], axis=1)
            temp_sort2 = temp2.sort_values(by='diff')
            temp_sort2 = temp_sort2.loc[(temp_sort2['SP']!=0)&(temp_sort2['LaneNo']!=1)&(temp_sort2['LaneNo']!=5)
                                        & (temp_sort2['LegNo']==4)& (temp_sort2['LaneType']==1)& (temp_sort2['SPValid']==1)]

        if ((in_queue>0) & (len(temp_sort2)>0)):
            if in_queue <= len(temp_sort2):
                delay_Q = temp_sort2.iloc[in_queue-1]['diff']
            else:
                delay_Q = temp_sort2.iloc[-1]['diff']

        # If there are no queued vehicles, no vehicles passing stop bar during next green phase, or both
        else: 
            delay_Q = 0
        # Average delay for the green interval
        delays = np.append(delays,delay_Q)
        
    # Delay distribution across different hours
    prev_idx = idx
    plt.figure(figsize = (10,6))
    ax = sns.distplot(delays, norm_hist = True)
    plt.xlabel("Delay (seconds)")
    plt.ylabel("Density")
    plt.title("Distribution of Delay at Hour " + hour)
    filename = 'Node' + node + 'Hour' + hour + '.png'
    plt.savefig(filename)
    
    # Save calculated delay
    delays_hr = pd.DataFrame([])
    delays_hr['Delay'] = delays
    filename = #Removed
    delays_hr.to_csv (filename, index = None, header=True)


In [ ]:
delays_hr = pd.DataFrame([])
delays_hr['Delay'] = delays
delays_hr.head()

In [ ]:
sns.distplot(delays)